<a href="https://colab.research.google.com/github/imji0319/new_python/blob/master/DACON/Arctic/shardcode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [DACON COMP : 북극 해빙 농도 예측 AI](https://dacon.io/competitions/official/235706/overview/) 





In [1]:
!pip install -q git+http://github.com/tensorflow/examples.git
!pip install -q tensorflow-addons

     |████████████████████████████████| 706kB 4.3MB/s 


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from pathlib import Path

import datetime
import os 
import time

import tensorflow as tf
from tensorflow_examples.models.pix2pix import pix2pix

In [3]:
print(f'tf.__version__ : {tf.__version__}')

tf.__version__ : 2.4.1


In [6]:
gpus = tf.config.list_physical_devices("GPU")

if gpus:
    try : 

        for gpu in gpus :
            tf.config.experimental.set_memory_growth(gpu, True)

        logical_gpus = tf.config.experimental.list_logical_devices("GPU")
        print(f"{len(gpus)} Physical GPUs, {len(logical_gpus)} Logical GPUs")

    except RuntimeError as e:
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [7]:
!nvidia-smi

Wed Mar  3 11:23:21 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P0    69W / 149W |    124MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------